<a href="https://colab.research.google.com/github/pakkei1212/SMU_AML_PROJ/blob/main/K_fold.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#@title connect google drive

from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/SMU_MITB_AML_PRO/

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/SMU_MITB_AML_PRO


In [2]:
from sklearn.model_selection import StratifiedKFold, cross_val_score, train_test_split

import numpy as np
import pandas as pd

from hotel_preprocessing import clean_data, extract_features, drop_columns, XYPreprocessor

In [3]:
raw_hotel_data = pd.read_csv("hotel_bookings.csv")
hotel_data = raw_hotel_data.copy()

In [10]:
# Combine year, month, and day columns into a single datetime column
hotel_data['arrival_date'] = pd.to_datetime(
    hotel_data['arrival_date_year'].astype(str) + '-' +
    hotel_data['arrival_date_month'] + '-' +
    hotel_data['arrival_date_day_of_month'].astype(str),
    format='%Y-%B-%d'  # Ensure proper month name parsing
)

# Determine the latest arrival date
last_date = hotel_data['arrival_date'].max()

# Define the cutoff date for the last 3 months
cutoff_date = last_date - pd.DateOffset(months=3)

# Split into test and train/val sets
test_data = hotel_data[hotel_data['arrival_date'] > cutoff_date].reset_index(drop=True)
train_val_data = hotel_data[hotel_data['arrival_date'] <= cutoff_date].reset_index(drop=True)

# Optional summary
print(f"Latest arrival date: {last_date.date()}")
print(f"Cutoff date for test set: {cutoff_date.date()}")
print(f"Train/Validation samples: {len(train_val_data)}")
print(f"Test samples: {len(test_data)}")

Latest arrival date: 2017-08-31
Cutoff date for test set: 2017-05-31
Train/Validation samples: 103505
Test samples: 15885


In [11]:
X_train_val_raw = train_val_data.drop(['is_canceled'], axis=1)
y_train_val_raw = train_val_data['is_canceled'].astype(int)

X_test_raw = test_data.drop(['is_canceled'], axis=1)
y_test_raw = test_data['is_canceled'].astype(int)

In [12]:
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=2025)
preprocessor = XYPreprocessor()

for train_idx, val_idx in skf.split(X_train_val_raw, y_train_val_raw):
    X_train_raw, X_val_raw = X_train_val_raw.iloc[train_idx], X_train_val_raw.iloc[val_idx]
    y_train_raw, y_val_raw = y_train_val_raw.iloc[train_idx], y_train_val_raw.iloc[val_idx]

    # Joint preprocessing of X and y
    X_train, y_train = preprocessor.fit_transform(X_train_raw, y_train_raw)
    X_val, y_val = preprocessor.transform(X_val_raw, y_val_raw)

    #print(X_train.head())
    #print(y_train.head())
    #print(X_val.head())
    #print(y_val.head())

In [13]:
 X_test, y_test = preprocessor.transform(X_test_raw, y_test_raw)

 print(X_test.head())
 print(y_test.head())

   lead_time  arrival_date_year  arrival_date_day_of_month  \
0   0.001357                1.0                        0.0   
2   0.024423                1.0                        0.0   
3   0.058345                1.0                        0.0   
4   0.017639                1.0                        0.0   
5   0.025780                1.0                        0.0   

   stays_in_weekend_nights  stays_in_week_nights    adults  children  babies  \
0                      0.0                  0.02  0.036364       0.0     0.0   
2                      0.0                  0.04  0.036364       0.0     0.0   
3                      0.0                  0.04  0.036364       0.0     0.0   
4                      0.0                  0.06  0.036364       0.0     0.0   
5                      0.0                  0.06  0.036364       0.0     0.0   

   is_repeated_guest  previous_cancellations  ...  country_binned_ITA  \
0                0.0                     0.0  ...                   0   
